# End to End Multiclass Dog Breed Classification 
 
This notebook builds an end to end multi-class image classifier using TensorFlow and TensorFlow Hub

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import tensorflow_hub as hub

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


### Prepping data: turning images into tensors

In [ ]:
# Take a look at labels
labels = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')

In [ ]:
labels.head()

In [ ]:
labels.describe()

In [ ]:
# How many images are there of each breed
labels['breed'].value_counts().plot.bar(figsize=(20,10))

In [ ]:
# On average, how many instances of each class?
labels['breed'].value_counts().median()

In [ ]:
# View an image 
from IPython.display import Image
Image('/kaggle/input/dog-breed-identification/train/09839ef1c5a5a5b3acb61c4093cab07f.jpg')

In [ ]:
# Make a list of filenames 
filenames = ['/kaggle/input/dog-breed-identification/train/' + fname + '.jpg' for fname in labels['id']]

In [ ]:
filenames[:10]

In [ ]:
import os
if len(os.listdir('/kaggle/input/dog-breed-identification/train/')) == len(filenames): 
    print('okay')
else: 
    print('not okay')

In [ ]:
# Convert labels to numpy array
labels_np = labels['breed'].to_numpy()

In [ ]:
labels_np[:10]

In [ ]:
# Check if length of filenames is same as length of labels_np
len(labels_np) == len(filenames)

In [ ]:
unique_labels = np.unique(labels_np)
unique_labels, len(unique_labels)

In [ ]:
# Convert unique labels into arrays of booleans 
bool_labels = [label == unique_labels for label in labels_np]
bool_labels[:2]

In [ ]:
print(bool_labels[0].astype(int))

In [ ]:
encoded_labels = [label.astype(int) for label in bool_labels]

In [ ]:
encoded_labels[:3]

In [ ]:
# Split into X and y
X = filenames 
y = encoded_labels

In [ ]:
X[:5]

In [ ]:
y[:5]

In [ ]:
NUM_IMAGES = 4000 #%param {type:"slider", min:1000, max:10000, step:1000} 
#only works in google colab


In [ ]:
# Split data into train and valid
from sklearn.model_selection import train_test_split


# Out of total number, NUM_IMAGES
X_train, X_valid, y_train, y_valid = train_test_split(X[:NUM_IMAGES], 
                                                      y[:NUM_IMAGES], 
                                                      test_size=0.2, 
                                                      random_state=42)

In [ ]:
len(X_train), len(X_valid)

Preprocessing images into Tensors

In [ ]:
# convert an image to np array
from matplotlib.pyplot import imread 
image = imread(filenames[42])
image.shape

In [ ]:
image[:2]

In [ ]:
tf.constant(image)[:2] # use tensorflow to convert image from np array to tensor

In [ ]:
# Function to preprocess image 
IMG_SIZE = 224

def process_image(img_path, img_size=IMG_SIZE): 
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
    
    return image

In [ ]:
# Fcn to return image and label as a tuple

def get_image_label(img_path, label): 
    image = process_image(img_path)
    return image, label

In [ ]:
process_image(X[42], tf.constant(y[42]))

In [ ]:
# Function to turn all data (X & y) into batches
# Define the batch size (default to 32)
BATCH_SIZE = 32

# Create a function to turn data into batches 
def create_data_batches(X, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False): 
    '''
    Creates batches of data out of image (X) and label (y) pairs.
    Shuffles the data if it's training data but doesn't shuffle if it's validation data.
    Also accepts test data as input (no lables).
    '''
    
    # If test dataset, no labels (only filepaths)
    if test_data: 
        print("Creating test data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X))) 
        # Process each image and add it to a batch
        data_batch = data.map(process_image).batch(BATCH_SIZE)
        return data_batch 
    
    # If valid dataset, no shuffling is necessary 
    elif valid_data: 
        print("Creating validation data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), #filepaths
                                                   tf.constant(y))) #labels
        data_batch = data.map(get_image_label).batch(BATCH_SIZE)
        return data_batch 
    # If training dataset, shuffle 
    else: 
        print("Creating training data batches...")
        # turn filepaths and labels into tensors
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), 
                                                  tf.constant(y)))  
        #Shuffling pathnames and labels before mapping image processor function is faster
        data = data.shuffle(buffer_size=len(X))
        
        # Create (image, label) tuples (and also turn image path into a preprocessed image)
        data = data.map(get_image_label)
        
        #Turn training data into batches
        data_batch = data.batch(BATCH_SIZE)
        
    return data_batch
                                                   
                                    

In [ ]:
# Create training and validation data batches
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_valid, y_valid, valid_data=True)

In [ ]:
# take a look at attributes of data batches
train_data.element_spec, val_data.element_spec

## Visualizing data batches 

In [ ]:
# create function for displaying images in a data batch, 25 images at a time
def show_25_images(images, labels): 
    plt.figure(figsize=(10,10))
    # displaying 25 images
    for i in range(25): 
        ax = plt.subplot(5, 5, i+1)
        plt.imshow(images[i])
        plt.title(unique_labels[labels[i].argmax()])
        plt.axis('off')

In [ ]:
# Unbatch data to visualize it 
train_images, train_labels = next(train_data.as_numpy_iterator())


In [ ]:
# Use function to visualize data in a training batch 
show_25_images(train_images, train_labels)

In [ ]:
# Visualization for validation set 
val_images, val_labels = next(val_data.as_numpy_iterator())
show_25_images(val_images, val_labels)

## Building a model

In [ ]:
# Setup shape of input
INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3] # for batch, height, width, color channel

# Setup output shape
OUTPUT_SHAPE = len(unique_labels)

# Set up model URL from TF Hub 
MODEL_URL = 'https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4'

In [ ]:
# Func to build Keras model 
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL): 
    print('Building model with:', MODEL_URL)
    
    # Set up the model layers
    model = tf.keras.Sequential([
        hub.KerasLayer(MODEL_URL), # Layer 1 (input layer)
        tf.keras.layers.Dense(units=OUTPUT_SHAPE,
                              activation='softmax') # Layer 2 (output layer)
    ])
    
    # Compile the model 
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['accuracy']
    )
    
    # Build the model 
    model.build(INPUT_SHAPE)
    
    return model

In [ ]:
model = create_model()
model.summary()

## TensorBoard callback 

### Setting TensorBoard up to work within Kaggle

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 
!mkdir ./logs/

In [ ]:
# Download Ngrok to tunnel the tensorboard port to an external port
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Run tensorboard as well as Ngrox (for tunneling as non-blocking processes)
import os
import multiprocessing


pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir ./logs/ --host 0.0.0.0 --port 6006 &",
                        "./ngrok http 6006 &"
                        ]]

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
# Set up TensorBoard callback 
import datetime 

def create_tensorboard_callback(): 
    logdir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    
    return tf.keras.callbacks.TensorBoard(logdir)

#### Early stopping callback

In [ ]:
# Preserve model's generalization (prevent overfitting) by early stopping callback
# Stop model if a certain eval metric stops improving 

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                  patience=3)


## Training a model (on subset of data)

Our first model is only going to train on 1000 images to make sure everything is working before we train on 10,000 images

In [ ]:
NUM_EPOCHS = 50 # Up to 100 chances to go through training set and learn patterns and make guesses


## DON'T FORGET TO MAKE SURE YOU ARE USING A GPU

In [ ]:
# function to train model 
def train_model(): 
    # Instantiate model
    model = create_model()
    
    #New tensorboard session whenever we train a model
    tensorboard = create_tensorboard_callback()
    
    # Fit model to data passing in the callbacks 
    model.fit(x=train_data,
              epochs=NUM_EPOCHS, 
              validation_data=val_data,
              validation_freq=1,
              callbacks=[tensorboard, early_stopping])
    
    # return fitted model 
    return model

# fit the model to the data
model = train_model()

### Checking the TensorBoard logs 

The TensorBoard Magic function will access the logs directory we created and visualize its contents 


In [ ]:
%tensorboard --logdir

## Making and evaluating predictions using a trained model

In [ ]:
val_data

In [ ]:
len(val_data)

In [ ]:
# make preds on the val data (not used to train on)
predictions = model.predict(val_data,verbose=1)
predictions 


In [ ]:
predictions.shape

In [ ]:
predictions[9]
# shows us a probability value for every single label 

In [ ]:
# first prediction 
index = 7
print(predictions[index])
print(f'Max value (probability of prediction): {np.max(predictions[index])}')
print(f'Sum: {np.sum(predictions[index])}')
print(f'Max index: {np.argmax(predictions[index])}')
print(f'Predicted label: {unique_labels[np.argmax(predictions[index])]}')

Predictions also have confidence intervals/ prediction probabilities.
* use everything over 75

We also want to see the image that the data is based on 

In [ ]:
# turn prediction probablities into their respective labels 

def get_pred_label(prediction_probabilities): 
    return unique_labels[np.argmax(prediction_probabilities)]


# Get a predicted label based on an array of prediction probabilities 
pred_label = get_pred_label(predictions[55])
pred_label 

### Unbatch data to make predictions on the validation images and then compare those predictions to the validation labels (truth labels)


In [ ]:

def unbatchify(data):
    '''Takes a batched dataset of (image, label) Tensors and returns separate arrays of images and labels'''
    images_ = []
    labels_ = []
    # loop through unbatched data 
    for image, label in data.unbatch().as_numpy_iterator():
        images_.append(image)
        labels_.append(unique_labels[np.argmax(label)])
        return images_, labels_
    
#Unbatchify the validation data
val_images, val_labels = unbatchify(val_data)
val_images[0]

In [ ]:
len(val_data)

In [ ]:
len(val_images)

In [ ]:
get_pred_label(val_labels[0])

## Visualizing Model Predictions 

Present it in a way that is usable to the user (e.g. if we are making the dog vision app) 



In [ ]:
# Function to visualize model predictions 

# Takes array of prediction probas, array of truth labels, array of images and integers
# convert prediction probas to a predicted label 
# plot predicted label, its predicted proba, the truth label, and target image on a single plot

def plot_pred(prediction_probabilities, labels, images, n=0): 
    pred_prob, true_label, image = prediction_probabilities[n], labels[n], images[n]
    
    # get the pred label 
    pred_label = get_pred_label(pred_prob)
    
    if pred_label ==true_label: 
        color='green'
    else: 
        color='red'
    
    # plot image and remove ticks 
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])
    
    # Change plot title to be predicted, probability of pred and truth label
    plt.title('{} {:2.0f}% {}'.format(pred_label, 
                                      np.max(pred_prob)*100,
                                      true_label), 
                                      color=color)

In [ ]:
len(val_images)

In [ ]:
plot_pred(prediction_probabilities=predictions, 
          labels=val_labels, 
          images=val_images, 
          n=0)

### compare first 10 predictions to truth 

Function will 
* Take an input of prediction probas array and a ground truth array and an integer
* Find the prediction using `get_pred_label()`
* Find the top 10 prediction probas indexes, pred proba values, pred labels
* Plot the top 10 probas values and labels, w/ true label colored green 

In [ ]:
def plot_pred_confidence(prediction_probabilities, labels, n=1): 
    pred_prob, true_label = prediction_probabilities[n], labels[n]
    
    # Get the predicted label
    pred_label = get_pred_label(pred_prob)
    
    # Find top 10 prediction confidence indexes
    top_10_pred_indexes = pred_prob.argsort()[-10:][::-1]
    #find the top 10 prediction confidence values
    top_10_pred_values = pred_prob[top_10_pred_indexes]
    #Find the top 10 prediction labels 
    top_10_pred_labels = unique_labels[top_10_pred_indexes]
    
    # set up plot 
    top_plot = plt.bar(np.arange(len(top_10_pred_labels)),
                       top_10_pred_values,
                       color='grey')
    plt.xticks(np.arange(len(top_10_pred_labels)),
               labels=top_10_pred_labels,
               rotation='vertical')
    
    # change color of true label 
    if np.isin(true_label, top_10_pred_labels): 
        top_plot[np.argmax(top_10_pred_labels == true_label)].set_color('green')
    else: 
        pass
    

In [ ]:
plot_pred_confidence(prediction_probabilities=predictions, 
                     labels=val_labels, 
                     n=0)

In [ ]:
# Image compared to top 10 pred confidences 
i_multiplier = 0
num_rows = 3
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(10*num_cols, 5*num_rows))

for i in range(num_images): 
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_pred(prediction_probabilities=predictions,
              labels=val_labels, 
              images=val_images, 
              n=i+i_multiplier)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_pred_confidence(prediction_probabilities=predictions, 
                   labels=val_labels, 
                   n=i+i_multiplier)
    plt.tight_layout(h_pad=1.0)
plt.show()

### Save and load trained model

In [ ]:
def save_model(model, suffix=None): 
    # Create a model directory pathname with current time (can't do that in Kaggle as far as I know)
    # modeldir = os.path.join('pathname/models',
                                #datetime.datetime.now().strftime('%Y%m%d-%H%M%s'))
        #model_path = modeldir + '-' + suffix + '.h5' # save model format
        model.save(model_path)
        return model_path

In [ ]:
def load_model(model_path):
    model = tf.keras.models.load_model(model_path, custom_objects={"KerasLayer": hub.KerasLayer})
    return model

In [ ]:
#Save model 
save_model(model, suffix='1000-images-mobilenetv2-Adam')

In [ ]:
# Evaluate the presaved model
model.evaluate(val_data)

## Training model on full dataset 

In [ ]:
len(X), len(y)

In [ ]:
# Create a data batch with the full data 
full_data = create_data_batches(X, y)

In [ ]:
full_data

In [ ]:
len(full_data)

In [ ]:
full_model = create_model()

In [ ]:
# Create full model callbacks 
# skip tensorboard for now 
# full_model_tensorboard = create_tensorboard_callback()

# No validation set when traiing on full data, so we can't monitor validation accuracy 

full_model_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy',
                                                             patience=3)

In [ ]:
# fit the full model to the full data 
full_model.fit(x=full_data,
               epochs=NUM_EPOCHS, 
               callbacks=[full_model_early_stopping])

In [ ]:
full_model.save('my_model.h5')

In [ ]:
loaded_full_model = tf.keras.models.load_model('my_model.h5', custom_objects={"KerasLayer": hub.KerasLayer})

In [ ]:
loaded_full_model.evaluate(val_data)

## Making predictions on test dataset

First must convert test data into the smae format at as the training data: 
1. Get the test image filenames
2. convert the filenames into test data batches using `create_data_batches()` and setting the `test_data` parameter to True (since the test data doesn't have labels)
3. Make a predictions array by passing the test batches to the predict() method colled on our model 

In [ ]:
# Load test image filenames 
test_path = '/kaggle/input/dog-breed-identification/test/'
test_filenames = [test_path + fname for fname in os.listdir(test_path)]
test_filenames[:10]

In [ ]:
len(test_filenames)

In [ ]:
# create test data batch 
test_data = create_data_batches(test_filenames, test_data=True)

In [ ]:
test_data

In [ ]:
len(test_data)

In [ ]:
# Make predictions using loaded full model 
test_predictions = loaded_full_model.predict(test_data, 
                                             verbose=1)

In [ ]:
test_predictions

In [ ]:
len(test_predictions)

In [ ]:
# save predictions to np arry 
np.savetxt('my_preds.csv', test_predictions, delimiter=',')

In [ ]:
test_preds = np.loadtxt('my_preds.csv', delimiter=',')

In [ ]:
test_preds

In [ ]:
test_preds_df = pd.DataFrame(test_preds)

In [ ]:
test_preds_df

In [ ]:
len(test_preds)

## Loading data into a sample submission 

In [ ]:
test_preds.shape